# Download do arquivo no S3

In [1]:
#Testing enviroment variable
import os
from dotenv import load_dotenv
load_dotenv()
name = os.getenv('NAME-TESTE')
print(name)

Victor Santos S


In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install psycopg2-binary

You are using pip version 9.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
#file download from S3
import boto3
from boto3 import client


def download_file(file_name, bucket='tekmore', new_name='livros_python.json'):
    s3 = boto3.client('s3')
    s3.download_file(bucket, file_name, new_name)

download_file('lista_livros_python.json')

In [13]:
#handling the file
import json
with open('livros_python.json') as arquivo:
    books = json.load(arquivo)
print(type(books))
print(type(books[0]))
print(books[1])
"""I have a dictionary list"""

<class 'list'>
<class 'dict'>
{'title_suggest': 'Pythons of Australia', 'edition_key': ['OL707632M'], 'isbn': ['0965744612', '9780965744614'], 'has_fulltext': False, 'text': ['OL707632M', '0965744612', '9780965744614', 'Brian A. Kend', 'Schuett, Laura White.', 'Schuett, Gordon W.', 'OL406588A', 'Pythons of Australia', 'Brian A. Kend ; edited by Gordon W. Schuett and Laura White Schuett.', 'Pythons', '/works/OL2765069W', 'Canyonlands Pub. Group', '97069689', 'Australia'], 'author_name': ['Brian A. Kend'], 'seed': ['/books/OL707632M', '/works/OL2765069W', '/subjects/pythons', '/subjects/place:australia', '/authors/OL406588A'], 'contributor': ['Schuett, Laura White.', 'Schuett, Gordon W.'], 'author_key': ['OL406588A'], 'subject': ['Pythons'], 'title': 'Pythons of Australia', 'publish_date': ['1997'], 'type': 'work', 'ebook_count_i': 0, 'publish_place': ['Provo, Utah'], 'edition_count': 1, 'key': '/works/OL2765069W', 'id_goodreads': ['3813466'], 'publisher': ['Canyonlands Pub. Group'], 'la

'I have a dictionary list'

In [14]:
#connectiong to postgresql database
import psycopg2
connection = psycopg2.connect(
    host='localhost',
    database='teste',
    user='vsantos93',
    password='swordfish'
)

cursor = connection.cursor()
#Creating tables
table_books = 'CREATE TABLE IF NOT EXISTS books (ID SERIAL PRIMARY KEY, KEY TEXT, TITLE TEXT, PUBLISHED_YEAR INT )'
table_subjects = 'CREATE TABLE IF NOT EXISTS books_subject (ID SERIAL PRIMARY KEY, BOOK_ID INT NOT NULL, NAME TEXT, FOREIGN KEY (BOOK_ID) REFERENCES BOOKS(ID) ON DELETE CASCADE)'
cursor.execute(table_books)
cursor.execute(table_subjects)
#Inserting a single value
cursor.execute('INSERT INTO books (ID, KEY, TITLE, PUBLISHED_YEAR) values (%s, %s, %s, %s)', (30, "VA", "Primeiro Teste", 1993))
connection.commit()

In [8]:
#inserting the values from books to database , a json object from file
for id, each_book in enumerate(books):
    """I am using the enumerate to generate a unique id for each book"""
    key = each_book['key']
    title = each_book['title']
    published_year = (each_book['publish_year'][0])
    subjects = each_book['subject']
    if published_year < 1991 or 'pythons' in title or 'snake' in title:
        pass
    else:
        cursor.execute('INSERT INTO books (ID, KEY, TITLE, PUBLISHED_YEAR) values (%s, %s, %s, %s)', (id, key, title, published_year))
    connection.commit()

In [82]:
for id, each_book in enumerate(books):
    key = each_book['key']
    title = each_book['title']
    published_year = (each_book['publish_year'][0])
    subjects = each_book['subject']
    if published_year < 1991 or 'pythons' in title or 'snake' in title or 'Pythons' in subjects:
        pass
    else:
        cursor.execute('INSERT INTO books (ID, KEY, TITLE, PUBLISHED_YEAR) values (%s, %s, %s, %s)', (id, key, title, published_year))
        for i, subject in enumerate(subjects):
            cursor.execute('INSERT INTO books_subject (book_id, name) values (%s, %s)', (id, subject))
    connection.commit()

In [81]:
#Trying to Remove the id manual insertion
for each_book in books:
    key = each_book['key']
    title = each_book['title']
    published_year = (each_book['publish_year'][0])
    subjects = each_book['subject']
    if published_year < 1991 or 'pythons' in title or 'snake' in title or 'Pythons' in subjects or 'pythons as pets' in subjects or 'Monty Python (Comedy troupe)' in subjects or 'Drama' in subjects:
        pass
    else:
        cursor.execute('INSERT INTO books (KEY, TITLE, PUBLISHED_YEAR) values (%s, %s, %s)', (key, title, published_year))
        for subject in subjects:
            cursor.execute('INSERT INTO books_subject (book_id, name) values (%s, %s)', (id, subject))
    connection.commit()
#crash

ForeignKeyViolation: insert or update on table "books_subject" violates foreign key constraint "books_subject_book_id_fkey"
DETAIL:  Key (book_id)=(99) is not present in table "books".


# Inserting the full file to database

In [86]:
for id, each_book in enumerate(books):
    key = each_book['key']
    title = each_book['title']
    published_year = (each_book['publish_year'][0])
    subjects = each_book['subject']
    cursor.execute('INSERT INTO books (ID, KEY, TITLE, PUBLISHED_YEAR) values (%s, %s, %s, %s)', (id, key, title, published_year))
    for i, subject in enumerate(subjects):
        cursor.execute('INSERT INTO books_subject (book_id, name) values (%s, %s)', (id, subject))
    connection.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


I could exchange the enumerate function to uuid lib to create an id numbers, but it seems to be more difficult than  the another one. 